In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Tue Jan 31 18:49:36 2017

@author: whikwon
"""

'''
Cross1 휘점 : 2305, 67841 
Cross1 쿠닉 : 2306, 67842
Cross1 군집S/C : 2307, 67843
Cross2 휘점 : 257, 65793
Cross2 쿠닉 : 258, 65794
Cross2 군집S/C : 259, 65795
정반사A 기포(백) : 1793, 67329
정반사A 이물(흑) : 1794, 67330
미분투과 이물 : 769, 66305
미분투과 라미눌림 : 770, 66306
미분투과 S/C : 772, 66308
정반사B 기포(백) : 1537, 67073
정반사B 이물(흑): 1538, 67074
정투과 점이물 : 1025, 66561
정투과 선이물 : 269, 66562
'''

import pymssql as pm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import QUERY_STATE as qs
import cx_Oracle
import AOI
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import os

os.chdir('C:\\Users\\whikwon\\Documents\\Python Scripts')
sns.set_style('whitegrid')

#font변경(LG스마트체)
font_path = 'C:/anaconda/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/LGSmHaR.ttf'
prop = font_manager.FontProperties(fname=font_path)
mpl.rcParams['font.family'] = prop.get_name()

#필요한 Database 불러오기
db1 = pm.connect('165.244.114.87', 'sa', '@admin123', 'LGCOPTMP')
dsn_tns = cx_Oracle.makedsn('165.244.121.4', 1521, 'IEPCS1')

db2 = cx_Oracle.connect('iepcs_view', 'viewdb7388', dsn_tns)

dsn_tns = cx_Oracle.makedsn('165.244.121.64', 3001, 'IEGOS')
db3 = cx_Oracle.connect('iegos_view', 'viewdb7388', dsn_tns)

In [ ]:
#Lot별 Data 불러오기 및 일부 정리
writer = pd.ExcelWriter('광학계 별 수율 확인.xlsx', engine = 'xlsxwriter')
workbook = writer.book
worksheet1 = workbook.add_worksheet('NBPC')
worksheet2 = workbook.add_worksheet('MOBILE')
worksheet3 = workbook.add_worksheet('AUTO')
worksheets = {'nbpc':worksheet1, 'mobile':worksheet2, 'auto':worksheet3}

#모델 구분, 정보, 폭 정보
# nbpc : LP156, mobile : Elsa, Auto : LP084 대표 모델 선정. 
model = {'nbpc' : ['RC40L25', 'RC40CLR', 'RC40HC', 'RC40KN16', 'ST46S25'], 'mobile' : ['PL23HC', 'GC20QHC', 'JC20HC'],'auto' : ['ZT40HC','ZT40CLR','ZT40SR02']}                  
model_size = {'nbpc' : [347.45, 196.8, 1402.8, 90], 'mobile' : [119.12, 67.38, 860, 0], 'auto' : [175.9, 132.8, 1245, 0]}
width_list = {'W1' : 1280, 'W3' : 1400, 'W4' : 1930, 'W5' : 2200}

data_start, date_end = '20170301', '20170426'

In [ ]:
for item in model.keys():
    
    for no, grade in enumerate(model[item]):
        lot_list = pd.read_sql_query(qs.find_das2(grade, date_start, date_end), db2)
        
        if len(lot_list) == 0:
            continue
        
        total = []
        total_2 = []
        grade = lot_list['PROD_CD'].iloc[0]
        
        for j, i in enumerate(lot_list['UNIQUE_LOT_NO']):

            lot = i
            width = width_list[pd.read_sql_query(qs.find_width(lot), db2).iloc[0][0][:2]]
            length = pd.read_sql_query(qs.find_length(lot), db2).iloc[0][0]
            data = pd.read_sql_query(qs.find_das(lot), db1)
            
            if ((len(data) == 0)):
                continue
            data.columns = ['불량명','Lot','카메라 번호', 'Size(max)','Size(min)','Value','불량번호','X','Y']
            data['Size'] = (data['Size(min)']+data['Size(max)'])/2
            data.dropna(subset = ['불량번호'], inplace = True)
            data['강/약'] = data['불량번호'].apply(lambda x: '강' if int(x)<60000 else '약') 
            data['Y'] = data['Y']/1000

            #이중마킹 고려
            if grade != 'PL23HC':
                data_1, data_2, data_3, data_4 = data.copy(), data.copy(), data.copy(), data.copy()         
                data_1['X'], data_1['Y'] = data_1['X'] + 12, data_1['Y'] - 12/1000
                data_2['X'], data_2['Y'] = data_2['X'] - 12, data_2['Y'] - 12/1000
                data_3['X'], data_3['Y'] = data_3['X'] + 12, data_3['Y'] - 12/1000
                data_4['X'], data_4['Y'] = data_4['X'] - 12, data_4['Y'] + 12/1000
                data_for_ratio = pd.concat([data, data_1, data_2, data_3, data_4], axis = 0)
            else:
                data_for_ratio = data
                
            #입력 값(인치 별로 나누는 함수)
            inch = AOI.inch_info(inch_x = model_size[item][0], inch_y = model_size[item][1], pitch = model_size[item][2], axis = model_size[item][2])
            inch_x, inch_y = inch[0], inch[1]
            
            #x,y bin 나누기 
            bin_x, bin_y = AOI.chip_division(width, inch_x), AOI.chip_division_y(length, inch_y)
            bin_y2 = AOI.chip_division_y(length, 100)                
            
            #Data 일부 수정
            data_for_ratio = AOI.revise_data(data_for_ratio, width, length, inch_x, inch_y)
            prod_ratio = AOI.prod_ratio3(data_for_ratio, bin_x, bin_y, lot)  
            total += prod_ratio
            
        #수율 관련 
        total = pd.DataFrame([total[0::4],total[1::4],total[2::4],total[3::4]], index = ['불량명','Chip개수','수율','Lot']).T
        total['수율'] = total['수율'].apply(lambda x: round(x, 2))
        total = total.groupby(['Lot','불량명'])[['수율']].sum().unstack()['수율']
        total = total.T[total.T != 0].dropna(how = 'all').T.fillna(0)
        
        #불량 별 색 지정 
        colors = {'크로스':'r','정투과':'c','미분투과':'g','정반사B':'y','투영':'gray', '크로스2':'orange', '정반사A' : 'm', '사선경계' : 'w'} 
        
        #Chip 기준으로 나누고 싶으면
        #==============================================================================
        # colors = {'chip1_x' : 'r', 'chip2_x' : 'b', 'chip3_x' : 'g', 'chip4_x' : 'y', 'chip5_x' : 'gray', 'chip6_x' : 'orange', 'chip7_x' : 'm', 'chip8_x' : 'w', 'chip9_x' : 'pink'}
        #==============================================================================
        value = []
        
        for i in range(0, len(total.T)-1):    
            value += [i for i in total.iloc[:, i]]
        bottom = [i/2 for i in total.iloc[:,0]]
        for i in range(1, len(total.T)-1):
                bottom += [i for i in total.iloc[:, :i].T.sum() + total.iloc[:,i]/2]
                           
        #Graph 그리기                 
        fig, ax = plt.subplots(1, 1, figsize = (20,12)) 
        for i in range(0, len(total.columns)-1):
            # 위치 조정
            ax.bar([i-0.25 for i in range(1, len(total)+1)], total.iloc[:,i], width = 0.5, bottom = total.iloc[:,:i].T.sum(), color = colors[total.columns[i]], label = total.columns[i]) # 변경
        
        # text 삽입
        rects = ax.patches
        for i, j in enumerate(rects):
            if i%2 == 0:
                ax.text(j.get_x() + j.get_width()/2, bottom[i], value[i], ha = 'center', va = 'center', fontsize = 13, color = 'k')
            else:    
                ax.text(j.get_x() + j.get_width()/2, bottom[i], value[i], ha = 'center', va = 'center', fontsize = 13, color = 'k')
        
        #전체 수율 삽입
        ax.plot([i for i in range(1, len(total)+1)],total.iloc[:,-1], marker = 'o', color = 'k')
        for i, j in enumerate(total.iloc[:,-1]):
            ax.text(i+0.75+rects[0].get_width()/2, 0.05+j, j, ha = 'center', color = 'k', fontsize = 13, fontweight = 'bold')
                
        ax.set_xlabel('Lot', size = 20)
        ax.set_xlim(0, len(total)+1)
        ax.set_ylabel('수율 감소(%)', size = 20)
        plt.xticks([i for i in range(1, len(total)+1)], total.index, rotation = 90, size = 15)
        plt.yticks(size = 15)
        plt.legend(loc = 'upper center',bbox_to_anchor=(0.5, 1.0), prop={'size':20}, ncol = len(total.columns))
        plt.grid(False)
        plt.tight_layout()
        plt.title('{}'.format(grade), size = 20)
        
        #Image 삽입
        if no%2 == 0:
            AOI.insert_image(fig, worksheets[item], 1+(no*11), 1, 0.35, 0.35)
        elif no%2 != 0:
            AOI.insert_image(fig, worksheets[item], 1+(no-1)*11, 12, 0.35, 0.35)
        
        #수율 계산 Data 삽임. 
        total.to_excel(writer,sheet_name =item+'_raw', startcol = no*10, startrow = 1)
  
#엑셀 닫기
workbook.close()    